In [28]:
import pandas as pd

In [29]:
# Reading the CSV files
df1 = pd.read_csv('Crash.csv')  # Load the Crash data from CSV
df2 = pd.read_csv('Units.csv', low_memory=False)  # Load the Units data from CSV
df3 = pd.read_csv('Casualty.csv')  # Load the Casualty data from CSV

In [30]:
# Merging datasets on common columns name of REPORT_ID
merged_data = pd.merge(df1, df2, on="REPORT_ID")
merged_data = pd.merge(merged_data, df3, on="REPORT_ID", how="outer")

In [31]:
print(merged_data.columns)

Index(['REPORT_ID', 'Stats Area', 'Suburb', 'Postcode', 'Total Units', 'Year',
       'Month', 'Day', 'Time', 'Area Speed', 'Position Type', 'Road Surface',
       'Moisture Cond', 'Weather Cond', 'DayNight', 'Crash Type',
       'CSEF Severity', 'Traffic Ctrls', 'Unit Type', 'Veh Year',
       'Licence Type', 'Rollover', 'Fire', 'Casualty Type', 'Sex', 'AGE',
       'Seat Belt', 'Helmet'],
      dtype='object')


In [32]:
# Replacing blank values (None or empty strings) with 'N' in the Rollover and Fire columns
columns_to_replace = ["Rollover", "Fire"]

# Replacing NaN with 'N'
merged_data[columns_to_replace] = merged_data[columns_to_replace].fillna("N")

# Replacing empty strings with 'N'
merged_data[columns_to_replace] = merged_data[columns_to_replace].replace("", "N")

print(merged_data['Fire'], merged_data['Rollover'])

0         N
1         N
2         N
3         N
4         N
         ..
147941    N
147942    N
147943    N
147944    N
147945    N
Name: Fire, Length: 147946, dtype: object 0         N
1         N
2         N
3         N
4         N
         ..
147941    N
147942    N
147943    N
147944    N
147945    N
Name: Rollover, Length: 147946, dtype: object


In [33]:
# Replacing empty strings with NaN for consistency
merged_data['Seat Belt'] = merged_data['Seat Belt'].replace("", None)

# Filling NaN in 'Seat belt' with corresponding values from 'Helmet'
merged_data['Seat Belt'] = merged_data['Seat Belt'].fillna(merged_data['Helmet'])

replace_dict = {
    "Worn": "Fitted - Worn",
    "Not Worn": "Fitted - Not Worn"
}

merged_data['Seat Belt'] = merged_data['Seat Belt'].apply(
    lambda x: replace_dict[x] if x in replace_dict else x
)

# Drop the 'Helmet' column as it's no longer needed
merged_data = merged_data.drop('Helmet', axis=1)

print(merged_data['Seat Belt'])

0                   NaN
1                   NaN
2                   NaN
3                   NaN
4                   NaN
              ...      
147941    Fitted - Worn
147942              NaN
147943              NaN
147944    Fitted - Worn
147945    Fitted - Worn
Name: Seat Belt, Length: 147946, dtype: object


In [34]:
# Defining the mapping from text to numeric values
severity_mapping = {
    '1: PDO': 1,   # Property Damage Only
    '2: MI': 2,    # Minor Injury
    '3: SI': 3,    # Serious Injury
    '4: Fatal': 4  # Fatality
}

# Create a new column with numeric values based on the severity_mapping
merged_data['severity_numeric'] = merged_data['CSEF Severity'].map(severity_mapping)

#made this column to check corealtions with other numeric values in the table

In [35]:
print(merged_data['severity_numeric'])

0         1.0
1         1.0
2         1.0
3         1.0
4         1.0
         ... 
147941    2.0
147942    1.0
147943    1.0
147944    2.0
147945    2.0
Name: severity_numeric, Length: 147946, dtype: float64


In [36]:
merged_data['AGE'] = pd.to_numeric(merged_data['AGE'], errors='coerce')

# Calculating the average of numeric values in 'Age', ignoring NaNs
age_avg = round(merged_data['AGE'].mean())  # Returns the mean, ignoring NaN values

# Replacing 'XX' and 'XXX' with the calculated average
merged_data['AGE'] = merged_data['AGE'].replace({'XX': age_avg, 'XXX': age_avg})

# Filling any remaining NaNs with the average
merged_data['AGE'] = merged_data['AGE'].fillna(age_avg)

In [37]:
# Converting 'Vehicle Year' to numeric, coercing errors to NaN
merged_data['Veh Year'] = pd.to_numeric(merged_data['Veh Year'], errors='coerce')

#The most frequent year (mode), ignoring NaN
most_frequent_year = merged_data['Veh Year'].mode().iloc[0]  # Mode returns a Series, get the first element


# Replace 'XXXX' with the most frequent year
merged_data['Veh Year'] = merged_data['Veh Year'].replace('XXXX', most_frequent_year)

# Fill remaining NaNs with the desired default (most frequent or average)
merged_data['Veh Year'] = merged_data['Veh Year'].fillna(most_frequent_year)

In [38]:
# Exporting merged data to a CSV file
merged_data.to_csv("merged_data.csv", index=False)